# Clase 5 – Enriquecimiento de la capa Plata

En esta notebook se


## Importar las librerías necesarias

In [30]:
import pandas as pd
from pathlib import Path

# Ajustar el ancho máximo para impresión en consola
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 300)         # Ajustar a un ancho suficiente en consola
pd.set_option('display.max_colwidth', None) # Evitar recortes en contenido de celdas

print("Importación de librerías completada.")

Importación de librerías completada.


## Configuración de paths y carpetas del proyecto

In [31]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'
PLATA_DIR = Path("../data/plata")

archivo_plata = PLATA_DIR / "misiones_plata.csv"
archivo_horario = PLATA_DIR / "misiones_horario.csv"

print("Iniciación de carpetas del proyecto completada.")

Iniciación de carpetas del proyecto completada.


## Carga del dataset y verificación de estructura

In [32]:
# Cargar el dataset diario
try:
    df_plata = pd.read_csv(archivo_plata, parse_dates=["FECHA"])
    print("Dataset diario cargado correctamente")
except FileNotFoundError:
    print("El archivo diario no fue encontrado")

# Cargar el dataset horario
try:
    df_horario = pd.read_csv(archivo_horario, parse_dates=["FECHA_HORA"])
    print("Dataset horario cargado correctamente")
except FileNotFoundError:
    print("El archivo horario no fue encontrado")

# Vista preliminar
print("\n Dataset diario:")
df_plata.info()
print("\n")
print(df_plata.head())

print("\n Dataset horario:")
df_horario.info()
print("\n")
print(df_horario.head())

✅ Dataset diario cargado correctamente
✅ Dataset horario cargado correctamente

📌 Dataset diario:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ESTACION              1095 non-null   object        
 1   FECHA                 1095 non-null   datetime64[ns]
 2   TEMP_MEAN             1095 non-null   float64       
 3   TEMP_MIN              1095 non-null   float64       
 4   TEMP_MAX              1095 non-null   float64       
 5   PNM_MEAN              1095 non-null   float64       
 6   PNM_MIN               1095 non-null   float64       
 7   PNM_MAX               1095 non-null   float64       
 8   HUM_MEAN              1095 non-null   float64       
 9   HUM_MIN               1095 non-null   int64         
 10  HUM_MAX               1095 non-null   int64         
 11  WIND_DIR_MEAN         1095 non-null 

Este paso permite validar la estructura general, tipos de datos y posibles columnas faltantes tanto en el dataset diario como en el horario. Si todo está correcto, avanzaremos con el enriquecimiento.

## Detección y análisis de fechas faltantes

Una vez verificada la estructura del dataset diario, procedemos a identificar si existen fechas faltantes en la serie por estación. 

Esto nos permitirá decidir estrategias para tratar los días sin registros, como imputación o exclusión.


In [34]:
# Generar el rango completo de fechas esperadas
fechas_totales = pd.date_range(start=df_plata['FECHA'].min(), end=df_plata['FECHA'].max(), freq='D')

# Obtener todas las combinaciones posibles de fecha y estación
estaciones = df_plata['ESTACION'].unique()
index_completo = pd.MultiIndex.from_product([fechas_totales, estaciones], names=['FECHA', 'ESTACION'])

# Reindexar para insertar NaNs explícitos en las fechas faltantes
df_plata = df_plata.set_index(['FECHA', 'ESTACION']).reindex(index_completo).reset_index()

# Verificar fechas faltantes (para exportar listado)
faltantes = df_plata[df_plata.isnull().any(axis=1)][['ESTACION', 'FECHA']]

if not faltantes.empty:
    faltantes.to_csv(PLATA_DIR / "fechas_faltantes.txt", index=False, sep='\t')
    print("Fechas faltantes exportadas a:", PLATA_DIR / "fechas_faltantes.txt")
else:
    print("No se encontraron fechas faltantes")

# Mostrar ejemplo si hay faltantes
print(faltantes.head())

📄 Fechas faltantes exportadas a: ../data/plata/fechas_faltantes.txt
    ESTACION      FECHA
4      OBERA 2024-06-02
25     OBERA 2024-06-09
46     OBERA 2024-06-16
88     OBERA 2024-06-30
109    OBERA 2024-07-07


Esta estrategia asegura que cada estación tenga una fila para cada fecha del rango, incluso si originalmente no había registros ese día. Esto deja los valores faltantes como `NaN`, que luego se tratarán.

## Tratamiento de valores nulos

Luego de verificar fechas faltantes, analizamos los valores `NaN` dentro del dataset actual para decidir estrategias de imputación o tratamiento.

### Tratamiento de datos faltantes en el dataset diario

In [35]:
# Visualizar cantidad de nulos por columna
print("\nValores nulos por columna:")
print(df_plata.isnull().sum())

# Calcular porcentaje de nulos por columna
porcentaje_nulos = df_plata.isnull().mean() * 100
print("\nPorcentaje de valores nulos:")
print(porcentaje_nulos.round(2))


Valores nulos por columna:
FECHA                    0
ESTACION                 0
TEMP_MEAN               90
TEMP_MIN                90
TEMP_MAX                90
PNM_MEAN                90
PNM_MIN                 90
PNM_MAX                 90
HUM_MEAN                90
HUM_MIN                 90
HUM_MAX                 90
WIND_DIR_MEAN           90
WIND_DIR_MIN            90
WIND_DIR_MAX            90
WIND_SPEED_MEAN         90
WIND_SPEED_MIN          90
WIND_SPEED_MAX          90
TEMP_MEAN_NORM          90
PNM_MEAN_NORM           90
HUM_MEAN_NORM           90
WIND_DIR_MEAN_NORM      90
WIND_SPEED_MEAN_NORM    90
dtype: int64

Porcentaje de valores nulos:
FECHA                   0.00
ESTACION                0.00
TEMP_MEAN               7.59
TEMP_MIN                7.59
TEMP_MAX                7.59
PNM_MEAN                7.59
PNM_MIN                 7.59
PNM_MAX                 7.59
HUM_MEAN                7.59
HUM_MIN                 7.59
HUM_MAX                 7.59
WIND_DIR_MEAN   

Una vez identificadas las columnas afectadas, proponemos distintas estrategias para completar los datos:

### Relleno con forward fill por estación

In [36]:
# Ordenar por estación y fecha para aplicar forward fill correctamente
df_plata_ffill = df_plata.sort_values(['ESTACION', 'FECHA']).copy()
df_plata_ffill.update(df_plata.groupby('ESTACION').ffill())

# Vista previa de ejemplo tras forward fill
print("\nEjemplo de datos tras forward fill:")
print(df_plata_ffill.head())


Ejemplo de datos tras forward fill:
        FECHA     ESTACION  TEMP_MEAN  TEMP_MIN  TEMP_MAX  PNM_MEAN  PNM_MIN  PNM_MAX  HUM_MEAN  HUM_MIN  HUM_MAX  WIND_DIR_MEAN  WIND_DIR_MIN  WIND_DIR_MAX  WIND_SPEED_MEAN  WIND_SPEED_MIN  WIND_SPEED_MAX  TEMP_MEAN_NORM  PNM_MEAN_NORM  HUM_MEAN_NORM  WIND_DIR_MEAN_NORM  WIND_SPEED_MEAN_NORM
0  2024-06-01  IGUAZU AERO       16.3      10.5      22.8    1019.3   1017.8   1021.6      82.4     62.0     95.0           89.2          50.0         110.0              8.8             4.0            17.0        0.405797       0.599303       0.707641            0.187275              0.256410
3  2024-06-02  IGUAZU AERO       19.7      14.0      28.8    1017.1   1015.0   1018.7      77.6     51.0     91.0           80.0          20.0         110.0              8.2             4.0            15.0        0.528986       0.522648       0.627907            0.159664              0.234432
6  2024-06-03  IGUAZU AERO       20.2      16.4      25.6    1019.3   1018.0   10

### Imputación con la media de cada estación (solo para columnas numéricas)

In [37]:
# Imputar con la media por estación
columnas_a_imputar = ['TEMP_MEAN', 'PNM_MEAN', 'HUM_MEAN', 'WIND_SPEED_MEAN', 'WIND_DIR_MEAN']

for col in columnas_a_imputar:
    df_plata_ffill[col] = df_plata_ffill.groupby('ESTACION')[col].transform(lambda x: x.fillna(x.mean()))

# Verificar resultado tras imputación
print("\nValores nulos después de imputación con medias:")
print(df_plata_ffill[columnas_a_imputar].isnull().sum())


Valores nulos después de imputación con medias:
TEMP_MEAN          0
PNM_MEAN           0
HUM_MEAN           0
WIND_SPEED_MEAN    0
WIND_DIR_MEAN      0
dtype: int64


Estas estrategias permiten garantizar que las variables derivadas a construir se basen en datos consistentes, sin afectar la distribución ni introducir sesgos evidentes.

### Tratamiento de datos faltantes en el dataset horario

In [48]:
# Detectar horarios reales de cada estación
df_horario['HORA'] = df_horario['FECHA_HORA'].dt.hour
horarios_por_estacion = df_horario.groupby('NOMBRE')['HORA'].value_counts().unstack(fill_value=0)
horarios_mas_frecuentes = horarios_por_estacion.idxmax(axis=1)

# Detectar horarios outlier (menos del 5% de los días)
outliers_horarios = {}
for estacion in horarios_por_estacion.index:
    total_dias = df_horario[df_horario['NOMBRE'] == estacion]['FECHA_HORA'].dt.date.nunique()
    outliers = horarios_por_estacion.loc[estacion][
        horarios_por_estacion.loc[estacion] / total_dias < 0.05
    ].index.tolist()
    if outliers:
        outliers_horarios[estacion] = outliers

# Crear index completo por estación y sus horarios típicos
df_horario['FECHA'] = df_horario['FECHA_HORA'].dt.floor('D')
estaciones_h = df_horario['NOMBRE'].unique()
fecha_h_min = df_horario['FECHA'].min()
fecha_h_max = df_horario['FECHA'].max()
rango_fechas = pd.date_range(start=fecha_h_min, end=fecha_h_max, freq='D')

# Crear combinaciones válidas por estación
porcentaje_frecuencia = 0.05 # al menos en 5% de los días

index_completo_personalizado = []
for estacion in estaciones_h:
    total_dias_estacion = df_horario[df_horario['NOMBRE'] == estacion]['FECHA'].nunique()
    horas_validas = horarios_por_estacion.columns[
        (horarios_por_estacion.loc[estacion] / total_dias_estacion) >= porcentaje_frecuencia  
    ].tolist()

    for fecha in rango_fechas:
        for hora in horas_validas:
            index_completo_personalizado.append((estacion, pd.Timestamp(fecha + pd.Timedelta(hours=hora))))

index_completo_h = pd.MultiIndex.from_tuples(index_completo_personalizado, names=['NOMBRE', 'FECHA_HORA'])

# Reindexar para insertar valores faltantes en los horarios esperados únicamente
df_horario_completo = df_horario.set_index(['NOMBRE', 'FECHA_HORA']).reindex(index_completo_h).reset_index()

# Verificación
print("\nDiferencia de tamaño (horas originales vs completadas por horario habitual):")
print("Original:", len(df_horario))
print("Completo:", len(df_horario_completo))
print("\nEjemplo de datos horarios con NaN insertados:")
print(df_horario_completo[df_horario_completo.isnull().any(axis=1)].head())




Diferencia de tamaño (horas originales vs completadas por horario habitual):
Original: 19656
Completo: 20145

Ejemplo de datos horarios con NaN insertados:
   NOMBRE          FECHA_HORA FECHA  HORA  TEMP  HUM  PNM  DD  FF  estacion_archivo
3   OBERA 2024-06-02 09:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
4   OBERA 2024-06-02 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
5   OBERA 2024-06-02 21:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
11  OBERA 2024-06-04 21:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
16  OBERA 2024-06-06 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN


### Mostrar horarios outliers detectados

In [53]:
# Visualizar registros reales en horarios atípicos detectados
print("\n Registros reales en horarios atípicos:")
for estacion, horas in outliers_horarios.items():
    print(f" - {estacion}: {horas}")

# Registrar los registros reales que ocurren en horarios atípicos
df_outliers_registros = []
for estacion, horas_outlier in outliers_horarios.items():
    registros_outlier = df_horario[
        (df_horario['NOMBRE'] == estacion) &
        (df_horario['HORA'].isin(horas_outlier))
    ]
    if not registros_outlier.empty:
        df_outliers_registros.append(registros_outlier)

# Concatenar y exportar si hay registros
if df_outliers_registros:
    df_outliers_concat = pd.concat(df_outliers_registros)
    archivo_outliers = PLATA_DIR / "registros_horarios_atipicos.csv"
    df_outliers_concat.to_csv(archivo_outliers, index=False)
    print("\n Archivo exportado con registros reales en horarios atípicos:")
    print(archivo_outliers)


 Registros reales en horarios atípicos:
 - OBERA: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23]

 Archivo exportado con registros reales en horarios atípicos:
../data/plata/registros_horarios_atipicos.csv


## Exportar datasets intermedios (antes de procesar los NaN)

In [55]:
# Exportar datasets intermedios (si se desea conservar)
df_plata.to_csv(PLATA_DIR / "misiones_plata_con_nan.csv", index=False)
df_plata_ffill.to_csv(PLATA_DIR / "misiones_plata_ffill.csv", index=False)
df_horario_completo.to_csv(PLATA_DIR / "misiones_horario_completo.csv", index=False)

print("Archivos generados correctamente")

Archivos generados correctamente


In [57]:
print(df_horario_completo)

            NOMBRE          FECHA_HORA      FECHA  HORA  TEMP   HUM     PNM     DD   FF  estacion_archivo
0            OBERA 2024-06-01 09:00:00 2024-06-01   9.0  16.8  72.0  1020.4  320.0  4.0        20240601.0
1            OBERA 2024-06-01 15:00:00 2024-06-01  15.0  22.8  84.0  1018.4   50.0  4.0        20240601.0
2            OBERA 2024-06-01 21:00:00 2024-06-01  21.0  16.8  63.0  1017.6   50.0  4.0        20240601.0
3            OBERA 2024-06-02 09:00:00        NaT   NaN   NaN   NaN     NaN    NaN  NaN               NaN
4            OBERA 2024-06-02 15:00:00        NaT   NaN   NaN   NaN     NaN    NaN  NaN               NaN
...            ...                 ...        ...   ...   ...   ...     ...    ...  ...               ...
20140  IGUAZU AERO 2025-06-30 19:00:00 2025-06-30  19.0   7.5  93.0  1028.5  230.0  4.0        20250630.0
20141  IGUAZU AERO 2025-06-30 20:00:00 2025-06-30  20.0   7.3  96.0  1028.9  230.0  4.0        20250630.0
20142  IGUAZU AERO 2025-06-30 21:00:00 2025-06

## Imputación de datos faltantes basada en promedio entre días anterior y posterior

In [61]:
# Variables a imputar
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

# Ordenar para asegurar coherencia
df_horario_completo = df_horario_completo.sort_values(by=['NOMBRE', 'FECHA_HORA'])

# Aplicar imputación: promedio entre valores del día anterior y posterior para el mismo horario
df_interp = df_horario_completo.copy()

for var in variables_objetivo:
    anterior = df_interp.groupby(['NOMBRE', df_interp['FECHA_HORA'].dt.hour])[var].shift(1)
    posterior = df_interp.groupby(['NOMBRE', df_interp['FECHA_HORA'].dt.hour])[var].shift(-1)

    # Calcular el promedio solo si ambos valores están presentes
    promedio = (anterior + posterior) / 2

    imputado = df_interp[var].copy()
    imputado = imputado.fillna(promedio)
    imputado = imputado.fillna(anterior)   # Si no hay ambos, usar anterior
    imputado = imputado.fillna(posterior)  # Si no hay anterior, usar posterior

    df_interp[var] = imputado

# Verificación de imputación final
print("Valores restantes faltantes por variable:")
print(df_interp[variables_objetivo].isnull().sum())

# Vista previa de algunos valores aún faltantes (si existen)
print("\nEjemplos de filas con valores aún faltantes:")
print(df_interp[df_interp[variables_objetivo].isnull().any(axis=1)].head())

# Guardar archivo imputado si lo deseás
# df_interp.to_csv(PLATA_DIR / "misiones_horario_imputado.csv", index=False)

Valores restantes faltantes por variable:
TEMP    76
HUM     76
PNM     76
DD      76
FF      76
dtype: int64

Ejemplos de filas con valores aún faltantes:
    NOMBRE          FECHA_HORA FECHA  HORA  TEMP  HUM  PNM  DD  FF  estacion_archivo
62   OBERA 2024-06-21 21:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
65   OBERA 2024-06-22 21:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
134  OBERA 2024-07-15 21:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
726  OBERA 2025-01-29 09:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
727  OBERA 2025-01-29 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN


In [60]:
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

df_interp = df_horario_completo.copy()

for var in variables_objetivo:
    anterior = df_interp.groupby('NOMBRE')[var].shift(1)
    posterior = df_interp.groupby('NOMBRE')[var].shift(-1)
    
    # Promedio cuando existen ambos
    promedio = (anterior + posterior) / 2

    # Asignar primero el promedio si ambos están presentes
    imputado = df_interp[var].copy()
    imputado = imputado.fillna(promedio)

    # Si sigue habiendo NaN, usar solo anterior si está
    imputado = imputado.fillna(anterior)

    # Si aún hay NaN, usar solo posterior
    imputado = imputado.fillna(posterior)

    df_interp[var] = imputado

# Verificar resultado de la imputación
print("Verificación de imputación entre días vecinos:\n")
print(df_interp[df_interp[variables_objetivo].isnull().any(axis=1)].head())
print("\n")
print(df_interp)

Verificación de imputación entre días vecinos:

   NOMBRE          FECHA_HORA FECHA  HORA  TEMP  HUM  PNM  DD  FF  estacion_archivo
4   OBERA 2024-06-02 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
25  OBERA 2024-06-09 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
46  OBERA 2024-06-16 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
66  OBERA 2024-06-23 09:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN
67  OBERA 2024-06-23 15:00:00   NaT   NaN   NaN  NaN  NaN NaN NaN               NaN


            NOMBRE          FECHA_HORA      FECHA  HORA  TEMP   HUM     PNM     DD   FF  estacion_archivo
0            OBERA 2024-06-01 09:00:00 2024-06-01   9.0  16.8  72.0  1020.4  320.0  4.0        20240601.0
1            OBERA 2024-06-01 15:00:00 2024-06-01  15.0  22.8  84.0  1018.4   50.0  4.0        20240601.0
2            OBERA 2024-06-01 21:00:00 2024-06-01  21.0  16.8  63.0  1017.6   50.0  4.0        20240601.0
3            OBERA 202